In [24]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import pickle


In [4]:
%pwd
%cd "../"
%cd "data"

c:\Users\johny\ChurnProject\Customer-Churn
c:\Users\johny\ChurnProject\Customer-Churn\data


In [5]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((5625, 35), (1407, 35), (5625, 1), (1407, 1))

In [7]:
# default parameters
model = GradientBoostingClassifier(random_state=420)
scoring = "accuracy"
model.fit(X_train, y_train)  # Train the model using the training data
predictions = model.predict(X_test)
# calculate accuracy 
accuracy = accuracy_score(y_test, predictions)




c:\Users\johny\anaconda3\envs\CustomerChurn\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
accuracy

0.7903340440653873

In [10]:
grid_parameters = {'n_estimators': [80, 90, 100, 110, 115, 120],
                   'max_depth': [3, 4, 5, 6],
                   'max_features': [None, 'auto', 'sqrt', 'log2'], 
                   'min_samples_split': [2, 3, 4, 5]}


# define the RandomizedSearchCV class for trying different parameter combinations
random_search = RandomizedSearchCV(estimator=GradientBoostingClassifier(),
                                   param_distributions=grid_parameters,
                                   cv=5,
                                   n_iter=150,
                                   n_jobs=-1)

# fitting the model for random search 
random_search.fit(X_train, y_train)

# print best parameter after tuning
print(random_search.best_params_)

c:\Users\johny\anaconda3\envs\CustomerChurn\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
155 fits failed out of a total of 750.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
106 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\johny\anaconda3\envs\CustomerChurn\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\johny\anaconda3\envs\CustomerChurn\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\johny\anaconda3\envs\CustomerChurn\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    vali

{'n_estimators': 115, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 3}


In [11]:
print(random_search.best_params_)

{'n_estimators': 115, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 3}


In [12]:
predictions = random_search.predict(X_test)
# calculate accuracy 
accuracy = accuracy_score(y_test, predictions)
accuracy

0.8024164889836531

In [18]:
confusion_matrix(y_test, predictions)


array([[924, 126],
       [152, 205]], dtype=int64)

In [23]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.86      0.88      0.87      1050
         1.0       0.62      0.57      0.60       357

    accuracy                           0.80      1407
   macro avg       0.74      0.73      0.73      1407
weighted avg       0.80      0.80      0.80      1407



In [27]:
filename = 'grad_boost_model.sav'
pickle.dump(random_search, open(filename, 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'model/grad_boost_model.sav'

In [29]:

%cd models

c:\Users\johny\ChurnProject\Customer-Churn\models


In [31]:
loaded_model = pickle.load(open("grad_boost_model.sav", 'rb'))
result = loaded_model.score(X_test, y_test)
result

0.8024164889836531